In [3]:
#CNN example pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN,super).__init__()
        self.conv1=nn.Conv2d(3,6,5)#input,output,m(mxm dimensions) kernel
        self.pool=nn.MaxPool2d(2,2)#reduce the size of the feature map to 2X2
        self.conv2=nn.Conv2d(6,16,5)#16 output channels
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)#10 output classes hence 10 neurons
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)#reshapes the data -1 lets the batch size be chosen by pytorch and 16*5*5 is the reshaped size
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        return F.softmax(self.fc3(x),dim=1)#softmax converts the raw values to probabilities hence returning the probabilities for all classes

In [8]:
#training using the CIFAR-10 dataset (defining the data loaders)
import torchvision
import torchvision.transforms as transforms
import torch

transform=transforms.Compose([
    transforms.ToTensor(),#images to range [0,1]
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))#normalize to range [-1,1]
])

trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)# to dowload training dataset
trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)#load the data in batches
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)#for test data
testloader=torch.utils.data.DataLoader(testset,batch_size=4,shuffle=True,num_workers=2)#dataloader for test data

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')